# Homework assignment 1: Markov Chain

## Problem 1
Read the article in https://www.geeksforgeeks.org/markov-chains-in-nlp/, and answer the following questions.

* (a) What are N-grams of an input sequence?
    * N-grams are a contiguous sequence of n items(words or characters) from a given sample of text.
    * For example, the input sequence "bong likes soccer", this is a 3-gram input.
    * N-grams in the example above are a contiguous sequences of n words, n is usually 2 or 3.

* (b) How to determine the probability of each element in the transition matrix?
    * By counting the number of times a particular element appears after another element in the N-grams.

* (c) If you want to increase the variety of the sequence generation (every time the outputs are different), what kinds of properties the training texts should be?
    * Increase the input length: This enables the output to have more variety, as it has multiple starting state points. This also increases the transistion probabilities.
    * Use multiple topics in text: This is done to diversify the text context and output.

## Problem 2
Try the following codes, and answer questions.

In [1]:
# Install the required packages
# !pip install nltk
# !pip install numpy

In [2]:
import re
from nltk import ngrams
import numpy as np
from typing import List, Tuple, Union, Dict
from itertools import product

In [3]:
def words_to_index(words: List[str], base: int, unique_words: Dict[str, int]) -> int:
    """
        Convert a list of words to an index representation.

        Args:
            words (List[str]): The list of words to be converted.
            base (int): The base value used for conversion.
            unique_words (Dict[str, int]): A dictionary mapping unique words to their corresponding indices.

        Returns:
            - int: The index representation of the given list of words.

        Example:
            >>> words_to_index(["the", "quick", "brown", "fox"], 10, {"the": 0, "quick": 1, "brown": 2, "fox": 3})
            123
    """
    length = len(words)
    numbers = [unique_words[word] for word in words]
    return sum([num * (base ** (length - 1 - idx)) for idx, num in enumerate(numbers)])

def index_to_words(index: int, base: int, length: int, unique_words: Dict[str, int]):
    """
        Converts an index to a list of words based on a given base and unique words.

        Args:
            index (int): The index to convert.
            base (int): The base used for conversion.
            length (int): The length of the resulting list.
            unique_words (Dict[str, int]): A dictionary mapping unique words to their corresponding indices.

        Returns:
            List[str]: A list of words corresponding to the given index.

        Examples:
            >>> unique_words = {'apple': 0, 'banana': 1, 'cherry': 2}
            >>> index_to_words(5, 3, 2, unique_words)
            ['banana', 'cherry'] (Because 5 = 1 * 3^1 + 2 * 2^0)

            >>> unique_words = {'red': 0, 'green': 1, 'blue': 2}
            >>> index_to_words(2, 3, 1, unique_words)
            ['blue'] (Because 2 = 2 * 3^0)
    """
    numbers = []

    unique_words_list = list(unique_words.keys())
    for pow in range(length - 1, -1, -1):
        numbers.append(index // (base ** pow))
        index -= numbers[-1] * (base ** pow)

    return [unique_words_list[num] for num in numbers]

### Step 1: Remove some unnecessary characters

In [4]:
def remove_unnecessary_characters(text: str):
    """
        Removes unnecessary characters from the given text and converts it to lowercase.

        Args:
            text (str): The input text to be processed.
        Returns:
            str: The processed text with unnecessary characters removed and converted to lowercase.
    """
    # Remove unnecessary characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Convert to lowercase
    text = text.lower()

    return text

### Step 2: Generate N-grams

In [5]:
def generate_n_grams(text: str, n: int):
    """
        Generate n-grams from the given text.

        Args:
            text (str): The input text from which n-grams will be generated.
            n (int): The number of consecutive words in each n-gram.
        Returns:
            List[Tuple[str]]: A list of tuples representing the generated n-grams.
    """
    # [TODO] Generate n-grams, using the ngrams function from the nltk library
    n_grams = ngrams(text.split(), n)

    # Convert to list of tuples
    n_grams = list(n_grams)

    return n_grams

### Step 3: Compute Transition Matrix

In [6]:
def compute_transition_matrix(n_grams: List[Tuple[str]]):
    """
        Computes the transition matrix and unique states for a given list of n-grams.

        Args:
            n_grams (List[Tuple[str]]): A list of n-grams, where each n-gram is a tuple of strings.
        Returns:
            np.ndarray: transition matrix.
            Dict[str, int]: A dictionary mapping unique words to their corresponding indices.
    """

    # Get the value of n
    n = len(n_grams[0])

    ## Step 3.1: Collect all possible words (label words with indices)

    unique_words = {}
    unique_words_count = 0

    # A helper function to add a word to the unique_words dictionary, if it is not already present
    def _add_to_unique_word(word: str):
        # We need to increase the counter outside this function
        nonlocal unique_words_count

        # Check if the state is already in the unique_states dictionary
        if unique_words.get(word, None) is None:
            unique_words[word] = unique_words_count
            unique_words_count += 1

    # Iterate over all n-grams to collect all possible words
    for n_gram in n_grams:
        for word in n_gram:
            # [TODO] Add the word to the unique_words dictionary
            # hint: use the _add_to_unique_word function
            _add_to_unique_word(word)
            continue


    ## Step 3.2: Compute the transition matrix

    # In this part, we use `words_to_index` and `index_to_words` functions to convert between state name (words) and indices

    # Example:
    # If we have 3 unique words: 0 for "apple", 1 for "banana", 2 for "cherry", and n = 3,
    # then the state names are "apple, apple", "apple, banana", "apple, cherry", "banana, apple", "banana, banana", and so on.
    # In this case, the mapping from states to indices would be:
    # "(apple, apple)" <-> 0, "(apple, banana)" <-> 1, "(apple, cherry)" <-> 2, "(banana, apple)" <-> 3, and so on.

    # [TODO] Compute the number of states in Markov chain. Number of states is the number of unique words to the power of n-1
    unique_states_count = unique_words_count ** (n-1)
    # [TODO] Create transition matrix, filled with zeros (the size of the transition matrix is # states x # states)
    transition_matrix = np.zeros((unique_states_count,unique_states_count))# pokoknya di isi kosong semua berdasarkan jumlah states

    # Count the number of transitions from each state to another state
    for n_gram in n_grams:
        # [TODO] Find the indices of the states (from and to) (i.e. the first n-1 words and the last n-1 words)
        # hint: use words_to_index function
        state_from_index = words_to_index(n_gram[:-1], unique_words_count, unique_words) #mulai dari elemen plg awal
        state_to_index = words_to_index(n_gram[1:], unique_words_count, unique_words) #sampe berakhir di elemen terakhir

        # Increment the count of the transition from state_from to state_to
        transition_matrix[state_from_index][state_to_index] += 1

    # Special case:
    # if the sum of number of transition of a state is zero, which means it has no information about the next state,
    # we set the probabilities of transitioning to all possible next states to 1 / # words

    # Check if any row sums to zero
    row_sums_is_zero = transition_matrix.sum(axis=1) == 0

    # Iterate over rows to find zero-sum rows
    first_word = list(unique_words.keys())[0]
    for idx, is_zero in enumerate(row_sums_is_zero):
        # If the row sum is zero
        if is_zero:
            # Convert the index to state representation
            state_name = index_to_words(idx, unique_words_count, n-1, unique_words)

            # Calculate the range of indices for the next posiible states
            possible_states_start_index = words_to_index(state_name[1:] + [first_word, ], unique_words_count, unique_words)
            possible_states_end_index = possible_states_start_index + unique_words_count

            # Set the number of transitions to 1 for the range
            transition_matrix[idx, possible_states_start_index : possible_states_end_index] = 1

    ## Step 3.3: Normalize the transition matrix

    # The following steps is to normalize transition matrix so that the sum of each row is 1
    # (The reason is: By the definition of a Markov chain, the sum of each row should be 1)

    # [TODO] Compute the sum of each row
    # hint: use np.sum function and check if the axis parameter is set correctly
    row_sums = np.sum(transition_matrix, axis=1)# karena nyari sum of each row pake 1, kalo nyari sum of each column baru pake 0

    # [TODO] Divide the transition matrix by the sum of each row to make the sum of each row equals to 1
    transition_matrix = transition_matrix / row_sums[:, np.newaxis] # intinya dibagi sum of each row biar sum of rownya jadi 1

    return transition_matrix, unique_words

### Step 4: Generate text

In [7]:
def generate(unique_words: Dict[str, int], transition_matrix: np.ndarray, start_from: Union[str, List[str]], n: int, length: int=10):
    """
        Generate text using a Markov chain model.

        Args:
            unique_states (Dict[str, int]): A dictionary mapping unique words to their corresponding indices.
            transition_matrix (np.ndarray): A numpy array representing the transition probabilities between states.
            start_from (Union[str, List[str]]): The starting state(s) for text generation.
            n (int): The size of the grams.
            length (int, optional): The length of the generated text. Defaults to 10.

        Returns:
            The generated text.
    """
    # Generate text
    generated_words = start_from.copy() if type(start_from) is list else [start_from]

    # Assert if the number of start words does not equal to n-1
    assert len(generated_words) >= n-1, "The number of start words should be greater than or equals to n-1 ({})".format(n-1)

    # Get the number of unique words
    unique_words_count = len(unique_words)

    # [TODO] Get the number of unique states
    # hint: check step 3.2 in the compute_transition_matrix function
    unique_states_count = unique_words_count ** (n-1)

    # Generate the next words
    for _ in range(length):
        # [TODO] Get index of current states
        # hint: The current states (current words) is the last n-1 words in the generated text
        # hint: use words_to_index function
        current_words_index = words_to_index(generated_words[-n+1:], unique_words_count, unique_words)

        # [TODO] Get probability distribution for next state, using the information in the transition matrix
        probabilities = transition_matrix[current_words_index]

        # Select next word based on probabilities, using np.random.choice function
        next_words_index = np.random.choice(unique_states_count, p=probabilities)

        # [TODO] Decode the index and get the last word
        # hint: use index_to_words function
        next_word = index_to_words(next_words_index, unique_words_count, n-1, unique_words)[-1]
        # because we use the first 2 words to predict the next word,
        # therefore when the index is converted back to word, it is n-1 length bcs we only use the first n-1 words in the list

        # Add next word to generated text
        generated_words.append(next_word)

    # return generated string
    return ' '.join(generated_words)

In [8]:
# [TODO] Change the text below and try different values of n
#text = "i love cat. cat love me. But I love dog too and dog also love me"
text="kevin lius bong. icho 3x winner. chemical engineering nus. hobi dia futsal. dia suka makan mie. dia suka minum matcha."
n = 4

# Process the text and generate the transition matrix
text = remove_unnecessary_characters(text)
n_grams = generate_n_grams(text, n)
transition_matrix, unique_words = compute_transition_matrix(n_grams)

In [9]:
# Print the transition matrix and unique states for obsevation
print("===== The indices for unique states are: =====")
unique_words_count = len(unique_words)
for word_name in list(product(*[unique_words for _ in range(n-1)]))[:20]:
    print(f"{','.join(word_name):10s}: {words_to_index(word_name, unique_words_count, unique_words)}")
print("...", end="\n\n")

print("===== The transition matrix is (Shape of trasition matrix: {}): =====".format(transition_matrix.shape))
print(transition_matrix)
print()

===== The indices for unique states are: =====
kevin,kevin,kevin: 0
kevin,kevin,lius: 1
kevin,kevin,bong: 2
kevin,kevin,icho: 3
kevin,kevin,3x: 4
kevin,kevin,winner: 5
kevin,kevin,chemical: 6
kevin,kevin,engineering: 7
kevin,kevin,nus: 8
kevin,kevin,hobi: 9
kevin,kevin,dia: 10
kevin,kevin,futsal: 11
kevin,kevin,suka: 12
kevin,kevin,makan: 13
kevin,kevin,mie: 14
kevin,kevin,minum: 15
kevin,kevin,matcha: 16
kevin,lius,kevin: 17
kevin,lius,lius: 18
kevin,lius,bong: 19
...

===== The transition matrix is (Shape of trasition matrix: (4913, 4913)): =====
[[0.05882353 0.05882353 0.05882353 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.05882353 0.05882353 0.05882353]]



In [13]:
# [TODO] Write down 3 or more initial words and length of generated text to start the text generation
experiments = [
    # Each entry is (Initial words, the length of the generated text)
   # ('i love', 10), ('cat love', 15), ('love me', 30)
    ('kevin lius bong', 15), ('hobi dia futsal', 20), ('chemical engineering nus', 13)
]

for idx, (start_from, length) in enumerate(experiments, 1):
    start_from = start_from.split(" ")

    # Generate text using the transition matrix
    generated_text = generate(unique_words, transition_matrix, start_from, n, length=length)

    # Print out the generated text
    print("#{} (length={}): {}".format(idx, length, generated_text))

#1 (length=15): kevin lius bong icho 3x winner chemical engineering nus hobi dia futsal dia suka makan mie dia suka
#2 (length=20): hobi dia futsal dia suka makan mie dia suka minum matcha hobi winner futsal bong futsal mie 3x suka bong suka nus kevin
#3 (length=13): chemical engineering nus hobi dia futsal dia suka makan mie dia suka minum matcha makan minum


### Answer the following questions

* (a) Write a new text of at least 15 words as the input.
    * kevin lius bong. icho 3x winner. chemical engineering nus. hobi dia futsal. dia suka makan mie. dia suka minum matcha.

* (b) Run the program 3 times with different output length and different initial words. Show the outputs.
    * #1 (length=15): kevin lius bong icho 3x winner chemical engineering nus hobi dia futsal dia suka makan mie dia suka
    * #2 (length=20): hobi dia futsal dia suka makan mie dia suka minum matcha hobi winner futsal bong futsal mie 3x suka bong suka nus kevin
    * #3 (length=13): chemical engineering nus hobi dia futsal dia suka makan mie dia suka minum matcha makan minum

* (c) Try different N of N-grams. How the N influences the output sequence?
    * By changing the N of N-grams, it influences the length of the context used to predict the next word. As It looks at (n-1) inputs to predict its next state.
    * For example,
      * 1st case: n=3, the inputs: ('kevin lius',5), ('matcha bong',3). The output will then be processed accordingly with no error, because the input is 2 which is exactly 3-1=2.
      * 2nd case: n=4, the previous input cannot be reused because now the n value is 4 so that means the inputs should contain 3 words as 4-1=3. For it to run properly, the input must be modified to ('kevin lius bong',5), ('dia suka makan',6).

## Problem 3
The Stationary Distribution of a Markov chain is a distribution of probabilities that remains unchanged after a transition from one state to another.

* (a) Ask an LLM (Large Language Model), such as ChatGPT, what are the applications of stationary distribution of a Markov chain. You need to show which prompts are used, and state how you verify the correctness of the results (output by LLMs).
    * I used Gemini AI to answer this question
    * This is the answer given:

      * Markov Chain Monte Carlo (MCMC):

        * Sampling from Complex Distributions: MCMC methods use Markov chains to generate samples from complex probability distributions that are difficult to sample directly. The stationary distribution of the Markov chain is designed to match the target distribution, ensuring that the generated samples are representative of the desired distribution.
        * Bayesian Inference: MCMC is widely used in Bayesian statistics to estimate parameters or posterior distributions. The stationary distribution of the MCMC algorithm provides the posterior distribution of interest.

      * Network Analysis:

        * PageRank Algorithm: Google's PageRank algorithm, used to rank web pages, is based on a Markov chain model. The stationary distribution of the chain represents the long-term probability of a random surfer ending up on a particular page, which is used to determine the page's importance.
        * Traffic Flow Analysis: Markov chains can be used to model traffic flow in networks, and the stationary distribution can provide insights into the long-term traffic patterns and congestion levels.

      * Biology and Genetics:

        * Genetic Drift: Markov chains are used to model genetic drift, the random fluctuations in allele frequencies in a population. The stationary distribution can be used to study the long-term effects of genetic drift on genetic diversity.
        * Phylogenetics: Markov chains are used to reconstruct evolutionary relationships between species (phylogenies). The stationary distribution of the Markov chain representing the evolutionary process is used to infer the most likely phylogeny.

      * Finance and Economics:

        * Financial Modeling: Markov chains are used to model the evolution of financial variables, such as stock prices or interest rates. The stationary distribution can provide insights into the long-term expected values and variability of these variables.
        * Credit Risk Assessment: Markov chains can be used to model the creditworthiness of borrowers. The stationary distribution can be used to assess the long-term probability of default and to develop risk management strategies.
  * The prompt used: What are the applications of stationary distribution of a Markov chain
  * I verify the results by cross-checking with the verified resources they enlist with the answer. In my case, i put the prompt on the google search bar and the list of references used was listed.

* (b) Ask an LLM, such as ChatGPT, what numerical method is the most efficient approach to compute the stationary distribution? You need to show which prompts are used, and state how you verify the correctness of the results (output by LLMs)
    * I used Gemini AI to answer this question
    * This is the answer given:
    * The most efficient numerical method to compute the stationary distribution of a Markov chain depends on the specific characteristics of the chain, such as its size, structure, and the desired level of accuracy. However, here are some commonly used methods:

      1. Power Method:

        This is a simple iterative method that involves repeatedly multiplying the initial probability distribution by the transition probability matrix.
        It converges to the stationary distribution as the number of iterations increases.
        The power method is generally efficient for small to medium-sized Markov chains.
      2. Jacobi or Gauss-Seidel Iteration:

        These methods solve the system of linear equations that define the stationary distribution.
        They are often more efficient than the power method for larger Markov chains.
        Jacobi iteration is a parallel algorithm, while Gauss-Seidel iteration is sequential.
      3. Successive Over-Relaxation (SOR):

        This is a variant of Gauss-Seidel iteration that can accelerate convergence by introducing a relaxation parameter.
        The optimal relaxation parameter can significantly improve the efficiency of SOR.
      4. Krylov Subspace Methods:

        These methods are based on projecting the problem onto a smaller subspace.
        They can be very efficient for large, sparse Markov chains.
        Examples of Krylov subspace methods include Arnoldi iteration and GMRES.
      5. Direct Methods:

        For small Markov chains, direct methods like Gaussian elimination can be used to solve the system of linear equations.
        However, direct methods can become computationally expensive for larger chains.
        The choice of method depends on factors such as the size of the Markov chain, the sparsity of the transition matrix, and the desired level of accuracy. For small to medium-sized Markov chains, the power method or Jacobi/Gauss-Seidel iteration are often sufficient. For larger, sparse chains, Krylov subspace methods can be more efficient. It's also important to consider the specific characteristics of the Markov chain and the computational resources available when selecting a method.
    * The prompt used: What numerical method is the most efficient approach to compute the stationary distribution?
    * I verify the results by cross-checking with the verified resources they enlist with the answer. In my case, i put the prompt on the google search bar and the list of references used was listed.
* (c) Implement the method suggested by the LLM. Use the transition matrix generated in question 2.a as an input to compute its stationary distribution.
    * Please implememt the method `compute_stationary_distribution` below
    * I used Chat GPT
* (d) The theory of probability matrix is given in the textbook 6.8, Eigenvalues/eigenvector of nonegtive matrices. Read textbook 6.8 and ask two questions that you are curious most about it
    * What are the real-life applications of Frobeinus Theorem? Is it more commonly applied then Perron's theorem?
    * What are the real-life applications of Perron's Theorem? Is it more commonly applied then Frobeinus theorem?

In [11]:
def compute_stationary_distribution(transition_matrix: np.ndarray):
    """
    Compute the stationary distribution of a Markov chain.

    Parameters:
        transition_matrix (np.ndarray): A numpy array representing the transition probabilities between states.
    Returns:
        The stationary distribution of the Markov chain.
    """
     # [TODO] Implement the function to compute the stationary distribution of a Markov chain, using the method suggested by the LLM
    # Ensure the transition matrix is a valid probability matrix
    assert np.allclose(transition_matrix.sum(axis=1), 1), "Rows of transition matrix must sum to 1"

    # Number of states
    n = transition_matrix.shape[0]

    # Solve the equation (P^T - I)v = 0
    A = np.transpose(transition_matrix) - np.eye(n)

    # Append a constraint to ensure the probabilities sum to 1
    A = np.vstack([A, np.ones(n)])
    b = np.zeros(n + 1)
    b[-1] = 1  # Constraint for the sum of probabilities

    # Use NumPy's least squares to find the stationary distribution
    stationary_distribution, _, _, _ = np.linalg.lstsq(A, b, rcond=None)

    # Normalize to ensure it's a probability distribution
    stationary_distribution /= stationary_distribution.sum()

    return stationary_distribution

In [12]:
# Compute the stationary distribution for the transition matrix obtained from the previous problem
stationary_distribution = compute_stationary_distribution(transition_matrix)

# Convert the stationary distribution to float16 to prevent something like -1.2345678e-16 but it is actually 0
print(stationary_distribution.astype(np.float16))

[0.0001978 0.0001978 0.0001978 ... 0.0002086 0.0002086 0.0002086]
